In [43]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
from ipywidgets import IntProgress
from IPython.display import display
import os
from IPython.display import clear_output
import json
from selenium.webdriver.support import expected_conditions
from json import JSONDecoder
from sc_header import aplanar, aplanar_lista,createDriver, extract_json_objects, scrape_url, scrape_batch, agregar_links

In [44]:
# def createDriver():

# user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'    

# chrome_options = Options()  
# chrome_options.add_argument("--headless")  
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument(f'user-agent={user_agent}')
# chrome_options.add_argument("--window-size=1825x1644")
# chromeOptions.addArguments("--disable-popup-blocking")
# chromeOptions.addArguments("--disable-notifications")
# chromeOptions.addArguments("--window-size=1920,1080")

# browser = webdriver.Chrome('/usr/bin/chromedriver', options = chrome_options)
browser = createDriver()
browser.delete_all_cookies()

In [45]:
url_base = 'https://www.shein.com/daily-new.html?cat_ids=2030,2032&ici=www_tab01navbar01&scici=navbar_2~~tab01navbar01~~1~~webLink~~~~0~~0'

browser.get(url_base)

In [46]:
#browser.save_screenshot('asd.png')


In [47]:
#CLICK EN MODAL
try:
    browser.find_element_by_css_selector('body > div.c-outermost-ctn > div.c-vue-coupon > div.coupon-mask.she-scroll-hide > div > i').click()
except:
    pass

In [48]:
#browser.find_element_by_css_selector('.nav2-sec-ctn.j-nav2-sec-ctn.j-sa-nav2-sec-ctn').click()

In [49]:
fecha = browser.find_element_by_css_selector('.filter-title.level-title.leftnav-second-title').find_element_by_tag_name('a')
fecha_alta = fecha.text
fecha.click()

In [50]:
women_element = browser.find_element_by_css_selector('.list-classify-filter.list-indent-one.j-fold-category-ctn').find_element_by_tag_name('li').find_element_by_tag_name('a')
women = women_element.text.upper()

In [51]:
#CLICK EN WOMEN
women_element.click()

In [52]:
category_list = browser.find_elements_by_css_selector('.list-classify-filter.list-indent-one')[1]

In [53]:
html_cat = []

for cat in category_list.find_elements_by_tag_name('li'):
    
    html_cat.append([cat.find_element_by_tag_name('a').get_attribute('href'),
                     cat.find_element_by_tag_name('a').get_attribute('data-cat-url-name')])
    

In [54]:
LINK_CATEGORIAS = [i for i in html_cat if i[1] in ['Clothing','Lingerie & Loungewear','Beachwear','Activewear']]

In [55]:
import pickle
def agregar_links(url_aux,driver,lista,tipologia_aux,url_concat):
    time.sleep(1)
    driver.get(url_aux)
#     pickle.dump(browser.get_cookies() , open("cookies.pkl","wb"))
#     cookies = pickle.load(open("cookies.pkl", "rb"))
#     for cookie in cookies:
#         browser.add_cookie(cookie)
    
    for elements in browser.find_elements_by_css_selector('.list-line-decorate.j-attr-filter'):
        
        if elements.find_element_by_tag_name('li').text == 'Color':
        #if elements.text == 'Color':
            print(tipologia_aux)

            for color in elements.find_elements_by_class_name('j-auto-attrlink'):

                lista.append([tipologia_aux,
                              color.get_attribute('data-attrvalue'),
                              color.get_attribute('href')])
                
    return lista

In [56]:
Tlistas = []
browser = createDriver()
url_base = 'https://www.shein.com'
for lsc in LINK_CATEGORIAS:
    
    agregar_links(lsc[0],
                 browser,
                 Tlistas,
                 lsc[1],
                 url_base)

Clothing
Lingerie & Loungewear
Beachwear
Activewear


In [57]:
#browser.delete_all_cookies()

In [58]:
hrefs_list = []
ex_col = []

for lista in Tlistas:
    
#for lista in Tlistas[4:]:

    try:
        browser.delete_all_cookies()
        browser.get(lista[2])
    except:
        pass
    
    while True:
        
        soup = BeautifulSoup(browser.page_source,'html.parser')

        indice_= 0
        
        if soup.select('div.j-switch-color-wrap > div.c-goodsitem__ratiowrap > a') == []: raise
            
        for element in soup.select('div.j-switch-color-wrap > div.c-goodsitem__ratiowrap > a'):
        #for element in soup.find_all(class_='c-goodsitem__ratioimg j-item-msg j-item-msg-a j-expose__target-goods-img'):
            
            
            href_aux = url_base + element.get('href')

            hrefs_list.append([indice_,
                               lista[0],
                               lista[1],
                               lista[2],
                               href_aux
                              ])
            indice_+=1
                
            #except:
            #    raise
            #    ex_col.append([lista[0],lista[2],lista[1]])
            #    pass            
        try:
            browser.find_element_by_class_name('page-next').find_element_by_tag_name('a').click()
        except:
            break
#browser.quit()


In [59]:
#soup.find_all(class_='c-goodsitem__ratioimg j-item-msg j-item-msg-a j-expose__target-goods-img')

In [60]:
df = pd.DataFrame(hrefs_list,columns=['pos','tipo','color','url_scraper','href'])

In [61]:
df['tipo'].unique()

array(['Clothing', 'Lingerie & Loungewear', 'Beachwear', 'Activewear'],
      dtype=object)

In [62]:
def check_length(serie):
    
    #if menor a 8 es porque hay un link donde el max es un numero que no es el id_producto
    ##caso especial : 'https://www.shein.com/skdress07191118781-p-929597-cat-2005.html?scici=homepage_164~~0_Banner_1_0_hotZone_w7bymbmsz~~3_2~~real_1991~~ccc_shein_pc_kids-homepage_default~~0~~50001'
    return str(max([X for X in map(int,re.findall(r"([\d]+)",serie)) if len(str(X))<8]))
    

In [63]:
df['auxi'] = df['href'].apply(check_length)

In [64]:
len(df.drop_duplicates()),len(df)

(562, 562)

In [65]:
#active = [i for i in Tlistas if i[0] == 'Activewear']

In [66]:
if len(df['tipo'].unique().tolist()) != len(set([item[0] for item in Tlistas])):
    print('REVISAR LOS TIPOS SCRAPEADOS')
    raise

In [67]:
len(df['tipo'].unique().tolist()),len(set([item[0] for item in Tlistas]))

(4, 4)

In [68]:
# def scrape_url(errores,items,url_aux,pos_aux,url_scraper,aux_tipologia,aux_color):
    
#     #lista_auxiliar = []
#     url = url_aux
#     for _ in range(5):
#         try:
#             txt = re.findall(r'productIntroData\s*:\s*({.*})', requests.get(url).text)
#             txt = txt[0]
#             break
#         except:
#             pass
#     try:
#         data = json.loads(txt)
#     except:
#         errores.append([url_aux,pos_aux,url_scraper,aux_tipologia,aux_color])
#         return 

#     #ID
#     codigo = data['detail']['goods_sn']
#     #DESCRIPCION
#     descripcion = data['detail']['goods_url_name']
#     #PRECIO 
#     precio = data['detail']['retailPrice']['amount']
#     #PRECIO CON DTO
#     precio_dto = data['detail']['salePrice']['amount']
#     #JPG
#     jpg = data['detail']['original_img']

#     for item in data['attrSizeList']:
        
#         items.append([pos_aux,
#                       aux_tipologia,
#                       aux_color,
#                       codigo,
#                       descripcion,
#                       item['attr_value_en'],
#                       item['stock'],
#                       precio,
#                       precio_dto,
#                       jpg,
#                       url_aux,
#                       url_scraper])
#     #return lista_auxiliar

# def scrape_batch(errores,items,url_chunk):
#     #chunk_resp = []
#     for indices,fila in url_chunk.iterrows():
#         scrape_url(errores,items,fila['href'],fila['pos'],fila['url_scraper'],fila['tipo'],fila['color'])
                     
#     #return chunk_resp

In [69]:
start_ = datetime.datetime.now()
items = []
errores = []

batch_size = 5
url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

var = 0
for ix, url_chunk in enumerate(url_chunks):
    scrape_batch(errores,items,url_chunk)
    
    if int(ix/50) == ix/50:
        
        try:
            df = pd.DataFrame(items)
            df.to_excel('lanzamiento.xlsx')
            
            if len(errores) >0:
                
                pd.DataFrame(errores).to_excel('lanzamiento_error.xlsx')
                
            file = open("lanzamiento.txt", "w")
            file.write( str(url_chunk) + os.linesep)
            file.close()
        except:
            print("Erro al crear .xlsx")    
    
end_ = datetime.datetime.now()

In [70]:
# start_ = datetime.datetime.now()
# items = []

# batch_size = 5
# url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

# var = 0
# for url_chunk in url_chunks:
#     items.append(scrape_batch(url_chunk))
#     var += 1
    
#     if var == 30:
#         var = 0
#         try:
#             df = pd.DataFrame(items)
#             df.to_excel('lanzamiento.xlsx')
            
                                                                                            
#             file = open("lanzamiento.txt", "w")
#             file.write( str(url_chunk) + os.linesep)
#             file.close()
#         except:
#             print("Erro al crear .xlsx")
        
    
# end_ = datetime.datetime.now()

In [71]:
print('Tiempo de ejecución: {}'.format(end_ - start_)[:-4])

Tiempo de ejecución: 0:55:39.61


In [72]:
# new_list = aplanar(items)

# new_list =[ i for i in new_list if type(i) != int]

In [73]:
df = pd.DataFrame(items)

In [74]:
df.rename(columns={0:'pos',
                  1:'tipo',
                  2:'color',
                  3:'sku',
                  4:'descripcion',
                  5:'talle',
                  6:'stock',
                  7:'precio',
                  8:'precio_dto',
                  9:'img',
                  10:'url',
                  11:'pagina_scraper'
                  },inplace=True)
df['fecha_alta'] = fecha_alta
df["marca"] = "SHEIN"
df["moneda"] = "DOLAR US$"

In [75]:
df["precio_dto"] = (df["precio_dto"]
                .str.extract(r"([\d,\.]+)", expand=False)
                .str.replace(",", ".")
                .astype(float))

In [76]:
df["precio"] = (df["precio"]
                .str.extract(r"([\d,\.]+)", expand=False)
                .str.replace(",", ".")
                .astype(float))

In [77]:
df['sexo'] ='Mujer'

In [78]:
df = df.drop_duplicates()

In [79]:
browser.quit()

In [80]:
df.dropna(inplace=True)

In [81]:
fecha = datetime.date.today()
df.to_excel('../Salida/lanzamiento'+str(fecha)+'.xlsx')
# writer = ExcelWriter('../Salida/lanzamiento'+str(fecha)+'.xlsx')
# df.to_excel(writer,'Hoja1')
# writer.save()

In [82]:
#INSERT
server = 'tcp:192.168.1.6'
database ="planeamiento"
username = 'sa'
password = 'sa'
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 

In [83]:
cursor = cnxn.cursor()
for index,row in df.iterrows():                                               
    try:
        cursor.execute("INSERT INTO sc_lanzamiento([sku],[pos],[talle],[color],[tipo],[marca],[sexo],[descripcion],[stock],[precio],[precio_dto],[fecha_alta],[moneda],[img],[url],[pagina_scraper]) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
                       row['sku'], 
                       row['pos'], 
                       row['talle'],
                       row['color'],
                       row['tipo'],
                       row['marca'],
                       row['sexo'],
                       row['descripcion'],
                       row['stock'],
                       row['precio'],
                       row['precio_dto'], 
                       row['fecha_alta'], 
                       row['moneda'],
                       row['img'], 
                       row['url'],
                       row['pagina_scraper']
                      ) 
        cnxn.commit()
    except Exception as e:
        
        print(e)
        print("*"*100)
        print(row)
        raise
cursor.close()
cnxn.close()

In [84]:
raise

RuntimeError: No active exception to reraise

In [ ]:
len(df)

In [ ]:
browser.quit()

In [ ]:
len(df)

In [ ]:
import pathlib
pathlib.Path().absolute()

In [ ]:
import os
os.getcwd()